In [ ]:
sql_str = '''
select 'Email' as InteractionType, emp.Contact_Id as Seller_Id, emp.Name as Seller, Emp_Title,
case when b.Contact_Id is not null then 1 else 0 end as PrimaryCaller_Flag,
c.Account_id, c.Account_Name,c.Contact_Id as ExtContact_Id,
c.Full_Name__c as ExtContact,Title,Account_Industry_Tier__c,
 EmailMinutes as Duration,Subject,Recipients_cnt as NoOfPeople,to_date(SentDate) as InteractionDate,
 to_date(SentDate) as EmailInteractionDate, Null as MeetingInteractionDate,
 case when year(SentDate) < 2017 then 2017 else year(SentDate) end as InteractionYear, year(current_date) as CurYear,
 case when Recipients_cnt > 10 then 10 else Recipients_cnt end as AdjustedNoOfPeople, Null as location_flag,
 0.5 as IntercationTypeF, 1 as location_flagF, 
(10 - (case when (year(current_date) - year(SentDate)) > 10 then 10 else  (year(current_date) - year(SentDate)) end))/10 as InteractionYearF
from 
(
    SELECT Sender,Recipient, 30 as EmailMinutes, Subject,Recipients_cnt, SentDate
    FROM messagesdata
) as e
inner join 
(select Id as Contact_Id, Name, Title as Emp_Title, lower(Email) as Email from Employee where Email is not null ) as emp on e.Sender = emp.Email
inner join
(
    SELECT Contact_Id, Account_id,Account_Name, Full_Name__c, Email,Title,Account_Industry_Tier__c
    FROM sellersextcontacts
) as c on e.Recipient = c.Email 
left join 
(
    SELECT Contact_Id, Full_Name__c, Email, Account_Id FROM PrimaryCallers where Email is not null
   --where Contact_Record_Type_Name__c = 'Employee'
) as b on e.Sender = b.Email and c.Account_id = b.Account_id
where 
lower(Subject) not like 're:%'
and Emp_Title in ('Director','Managing Director','Senior Director','Vice President')
and to_date(SentDate) >= add_months(current_date,-60)

UNION ALL

select 'Meeting' as InteractionType, emp.Contact_Id as Seller_Id, emp.Name as Seller, Emp_Title,
case when b.Contact_Id is not null then 1 else 0 end as PrimaryCaller_Flag,
c.Account_id, c.Account_Name,c.Contact_Id as ExtContact_Id,
c.Full_Name__c as ExtContact,Title,Account_Industry_Tier__c,
 MeetingMinutes as Duration,Subject,Attendees_cnt as NoOfPeople,to_date(StartDate) as InteractionDate,
 Null as EmailInteractionDate, to_date(StartDate) as MeetingInteractionDate,
 case when year(StartDate) < 2017 then 2017 else year(StartDate) end as InteractionYear, year(current_date) as CurYear,
case when Attendees_cnt > 10 then 10 else Attendees_cnt end as AdjustedNoOfPeople,location_flag,
1 as IntercationTypeF, case when location_flag = 'Remote' then 0.5 else 1 end as location_flagF, 
(10 - (case when (year(current_date) - year(StartDate)) > 10 then 10 else  (year(current_date) - year(StartDate)) end))/10 as InteractionYearF
from 
(
    SELECT Organizer,Attendee,(unix_timestamp(End)-unix_timestamp(Start))/(60) as MeetingMinutes, Subject,Attendees_cnt, Start as StartDate,
    case when location_flag is null then 'InPerson' else location_flag end as location_flag 
    FROM eventsdata
) as e
inner join 
(select Id as Contact_Id, Name, Title as Emp_Title, lower(Email) as Email from employee where Email is not null ) as emp on e.Organizer = emp.Email
inner join
(
    SELECT Contact_Id, Account_id,Account_Name, Full_Name__c, Email,Title,Account_Industry_Tier__c
    FROM sellersextcontacts
) as c on e.Attendee = c.Email 
left join 
(
    SELECT Contact_Id, Full_Name__c, Email, Account_Id FROM PrimaryCallers where Email is not null
   --where Contact_Record_Type_Name__c = 'Employee'
) as b on e.Organizer = b.Email and c.Account_id = b.Account_id
where 
lower(Subject) not like 're:%'
and Emp_Title in ('Director','Managing Director','Senior Director','Vice President')
and to_date(StartDate) >= add_months(current_date,-60)
and to_date(StartDate) <= current_date
'''

df_interactions = spark.sql(sql_str)
df_interactions.write.mode("overwrite").saveAsTable("interactions")   

In [ ]:
sql_str = '''select Seller_Id, ExtContact_Id, Account_Id,InteractionType,InteractionDate,
EmailInteractionDate,MeetingInteractionDate,
PrimaryCaller_Flag, Emp_Title,
sum(Duration) as Duration,
round(sum(Duration * IntercationTypeF * location_flagF * InteractionYearF),2) as AdjustedDuration, 
round(sum(Duration)/60/24,2) as Interaction, 
count(*) as NoOfInteractions from interactions
where InteractionDate >= add_months(current_date,-60)
and InteractionDate <= current_date
group by Seller_Id, ExtContact_Id, Account_Id,InteractionType,InteractionDate,EmailInteractionDate, MeetingInteractionDate, PrimaryCaller_Flag,Emp_Title'''

df_interactionsdata = spark.sql(sql_str)
df_interactionsdata.write.mode("overwrite").saveAsTable("interactionsdata")

In [ ]:
from pyspark.sql.functions import *
sql_str = '''select Seller_Id, Account_Id,
round(sum(Duration * IntercationTypeF * location_flagF * InteractionYearF),2) as AdjustedDuration,
max(EmailInteractionDate) as LastEmailInteration, max(MeetingInteractionDate) as LastMeetingInteraction,
datediff(current_date,max(EmailInteractionDate)) as LastEmailInterationDays, 
datediff(current_date,max(MeetingInteractionDate)) as LastMeetingInteractionDays,
case when datediff(current_date,max(EmailInteractionDate)) < 180 then 0 else 1 end as email_na_flag,
case when datediff(current_date,max(MeetingInteractionDate)) < 180 then 0 else 1 end as meeting_na_flag,
case when datediff(current_date,max(MeetingInteractionDate)) < 180 or datediff(current_date,max(MeetingInteractionDate)) < 180 then 0 else 1 end as ineteraction_na_flag
from
(
select Seller_Id, Account_Id,IntercationTypeF,location_flagF, InteractionYearF,Duration,
case when InteractionType = 'Email' then InteractionDate else null end as EmailInteractionDate,
case when InteractionType = 'Meeting' then InteractionDate else null end as MeetingInteractionDate
from interactions
where PrimaryCaller_Flag = 1
and InteractionDate <= current_date
and InteractionDate >= add_months(current_date,-60)
and Emp_Title in ('Director','Managing Director','Senior Director','Vice President')
)
group by Seller_Id, Account_Id'''

df_interactionscores = spark.sql(sql_str)
max_value=df_interactionscores.agg(max("AdjustedDuration")).collect()[0][0]
df_interactionscores = df_interactionscores.withColumn("MaxValue",lit(max_value))

df_interactionscores = df_interactionscores.withColumn("ImpactScore", round(col('AdjustedDuration')/col('MaxValue') * 100,2))

df_interactionscores.write.mode("overwrite").saveAsTable("impactscores") 

In [ ]:
from pyspark.sql.functions import *
sql_str = '''
select acc.Id as Account_Id, 'Met with CEO in last 6 months' as CheckListType, 
case when Flag_Value is null then 0 else Flag_Value end as Flag_Value
from Account as acc 
left join 
(
    SELECT
    a.Id as Account_Id,
    case when datediff(current_date, max(InteractionDate)) < 180 then 1 else 0 end as Flag_Value
    FROM  Account as a
    inner join Interactions as i on a.Id = i.Account_Id
    inner join Contact as c on i.ExtContact_Id = c.Id
    where upper(c.Title) like '%CEO%'
    and a.Tier in ('Tier 0','Tier 1','Tier 2')
    and InteractionType = 'Meeting'
    and PrimaryCaller_Flag = 1
    and InteractionDate >= add_months(current_date,-60)
    Group BY a.Id
) as res1 
on acc.Id = res1.Account_Id

UNION ALL 

select acc.Id as Account_Id, 'Met with CFO in last 6 months' as CheckListType, 
case when Flag_Value is null then 0 else Flag_Value end as Flag_Value
from Account as acc 
left join 
(
    SELECT
    a.Id as Account_Id,
    case when datediff(current_date, max(InteractionDate)) < 180 then 1 else 0 end as Flag_Value
    FROM  Account as a
    inner join Interactions as i on a.Id = i.Account_Id
    inner join Contact as c on i.ExtContact_Id = c.Id
    where upper(c.Title) like '%CFO%'
    and a.Tier in ('Tier 0','Tier 1','Tier 2')
    and InteractionType = 'Meeting'
    and PrimaryCaller_Flag = 1
    and InteractionDate >= add_months(current_date,-60)
    Group BY a.Id
) as res1 
on acc.Id = res1.Account_Id

UNION ALL

select acc.Id as Account_Id, 'Met with CIO in last 6 months' as CheckListType, 
case when Flag_Value is null then 0 else Flag_Value end as Flag_Value
from Account as acc 
left join 
(
    SELECT
    a.Id as Account_Id,
    case when datediff(current_date, max(InteractionDate)) < 180 then 1 else 0 end as Flag_Value
    FROM  Account as a
    inner join Interactions as i on a.Id = i.Account_Id
    inner join Contact as c on i.ExtContact_Id = c.Id
    where upper(c.Title) like '%CIO%'
    and a.Tier in ('Tier 0','Tier 1','Tier 2')
    and InteractionType = 'Meeting'
    and PrimaryCaller_Flag = 1
    and InteractionDate >= add_months(current_date,-60)
    Group BY a.Id
) as res1 
on acc.Id = res1.Account_Id
'''

df_pchecklist = spark.sql(sql_str)

df_pchecklist.write.mode("overwrite").saveAsTable("prospectingchecklist") 